<a href="https://colab.research.google.com/github/cdbolivarz/Accidentality_Predictions_Medellin/blob/master/Limpieza__y_formato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y formato de datos
En este documento, se realiza el preprocesamiento de los datos obtenidos de los documentos de incidentes georeferenciados.

Librerias

In [21]:
import pandas as pd
import glob
import numpy as np

### Lectura de datos

In [66]:
all_files = glob.glob("dataset/I*.csv")

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    li.append(frame)

df = pd.concat(li, axis=0, ignore_index=True)
df = df.sort_values(by=["FECHA"])
df.tail()

,X,Y,OBJECTID,RADICADO,FECHA,HORA,DIA,PERIODO,CLASE,DIRECCION,DIRECCION_ENC,CBML,TIPO_GEOCOD,GRAVEDAD,BARRIO,COMUNA,DISENO,DIA_NOMBRE,MES,MES_NOMBRE,X_MAGNAMED,Y_MAGNAMED,LONGITUD,LATITUD
136170,-75.543272,6.263863,546666,1611530,2017/12/31 00:00:00+00,09:30:00,31,2017,Atropello,CL 69 A CR 30,CL 069 A 030 000 00000,0309,Malla vial,HERIDO,Versalles No. 1,Manrique,Tramo de via,DOMINGO,12,NaN,837771.13,1184650.17,-75.543272,6.263863
136171,-75.579780,6.295075,546667,1611706,2017/12/31 00:00:00+00,09:30:00,31,2017,Volcamiento,CL 97 CR 76,CL 097 076 000 00000,0606,Malla vial,HERIDO,San Martín de Porres,Doce de Octubre,Lote o Predio,DOMINGO,12,NaN,833730.27,1188102.60,-75.579780,6.295075
135651,-75.552694,6.263473,546147,1611572,2017/12/31 00:00:00+00,03:00:00,31,2017,Choque,CR 42 CL 68,CR 042 068 000 00000,0308,EPM con Interior,HERIDO,Manrique Oriental,Manrique,Tramo de via,DOMINGO,12,NaN,836728.21,1184606.97,-75.552694,6.263473
136227,-75.605985,6.231864,546723,1611528,2017/12/31 00:00:00+00,10:25:00,31,2017,Atropello,CR 83 CL 30 A,CR 083 030 A 000 00000,1603,Malla vial,HERIDO,Belén,Belén,Interseccion,DOMINGO,12,NaN,830829.43,1181110.77,-75.605985,6.231864
136169,-75.559385,6.203245,546665,1611607,2017/12/31 00:00:00+00,08:30:00,31,2017,Caida Ocupante,CR 29 CL 6,CR 029 006 000 00000,1409,Malla vial,HERIDO,El Tesoro,El Poblado,Lote o Predio,DOMINGO,12,NaN,835987.71,1177944.91,-75.559385,6.203245


Que columnas tienen valores nulos

In [25]:
df.isna().any()

X                False
Y                False
OBJECTID         False
RADICADO         False
FECHA            False
HORA             False
DIA              False
PERIODO          False
CLASE             True
DIRECCION        False
DIRECCION_ENC     True
CBML              True
TIPO_GEOCOD      False
GRAVEDAD         False
BARRIO            True
COMUNA            True
DISENO            True
DIA_NOMBRE       False
MES              False
MES_NOMBRE        True
X_MAGNAMED       False
Y_MAGNAMED       False
LONGITUD         False
LATITUD          False
dtype: bool

Formato datos

In [68]:
#Eliminar filas con NA y columnas que no consideramos importantes para el modelo.
#Asignar mismos nombres a los tipos de accidentes (CLASE)
def format(df):
  df = df.drop(['X', 'Y', 'OBJECTID', 'RADICADO', 'DIRECCION', 'DIRECCION_ENC', 'FECHA',
              'CBML', 'TIPO_GEOCOD', 'DISENO', 'MES_NOMBRE', 'X_MAGNAMED',
              'Y_MAGNAMED'], axis=1)
  df = df.dropna(subset=['CLASE', 'BARRIO', 'COMUNA'])

  df["CLASE"].replace({"Caída de Ocupante": "Caida Ocupante",
                       "Caida de Ocupante": "Caida Ocupante",
                       "Caída Ocupante": "Caida Ocupante"}, inplace=True)

  return df

df = format(df)
df.head()

,HORA,DIA,PERIODO,CLASE,GRAVEDAD,BARRIO,COMUNA,DIA_NOMBRE,MES,LONGITUD,LATITUD
0,02:20:00,1,2014,Choque,HERIDO,Loma de los Bernal,Belén,MIÉRCOLES,1,-75.602726,6.219016
62,04:20:00,1,2014,Choque,HERIDO,Campo Amor,Guayabal,MIÉRCOLES,1,-75.579329,6.209160
61,09:10:00,1,2014,Choque,HERIDO,Barrio Colón,La Candelaria,MIÉRCOLES,1,-75.565164,6.242722
60,08:50:00,1,2014,Choque,HERIDO,San Miguel,Villa Hermosa,MIÉRCOLES,1,-75.555400,6.252163
59,08:00:00,1,2014,Otro,HERIDO,La Candelaria,La Candelaria,MIÉRCOLES,1,-75.564683,6.249282


Filas con errores ortograficos en algunas columnas

In [74]:
df['COMUNA'].drop_duplicates()

0                                              Belén
62                                          Guayabal
61                                     La Candelaria
60                                     Villa Hermosa
58                                      Buenos Aires
56                                           Popular
55                                          Aranjuez
64                                           Robledo
48                                  Laureles Estadio
47                                   Doce de Octubre
84                                          Manrique
81                                          Castilla
9                                         El Poblado
20             Corregimiento de San Antonio de Prado
36                                        San Javier
34                                        La América
23                    Corregimiento de San Cristóbal
123                                       Santa Cruz
184                       Corregimiento de Alt

In [75]:
df['BARRIO'].drop_duplicates()

0                               Loma de los Bernal
62                                      Campo Amor
61                                    Barrio Colón
60                                      San Miguel
59                                   La Candelaria
58                                    La Milagrosa
56                                       San Pablo
55                                        Brasilia
64                                     Villa Flora
49                                      Aures No.1
48                                   La Castellana
47                                         Kennedy
65                                 Bello Horizonte
67                            Alejandro Echavarría
85                                     La Pilarica
84                                 Versalles No. 1
83                              Campo Valdés No. 2
82                                    La Hondonada
81                                         Toscana
80                             

In [63]:
df.head()

,HORA,DIA,PERIODO,CLASE,GRAVEDAD,BARRIO,COMUNA,DIA_NOMBRE,LONGITUD,LATITUD
0,02:20:00,1,2014,Choque,HERIDO,Loma de los Bernal,Belén,MIÉRCOLES,-75.602726,6.219016
62,04:20:00,1,2014,Choque,HERIDO,Campo Amor,Guayabal,MIÉRCOLES,-75.579329,6.209160
61,09:10:00,1,2014,Choque,HERIDO,Barrio Colón,La Candelaria,MIÉRCOLES,-75.565164,6.242722
60,08:50:00,1,2014,Choque,HERIDO,San Miguel,Villa Hermosa,MIÉRCOLES,-75.555400,6.252163
59,08:00:00,1,2014,Otro,HERIDO,La Candelaria,La Candelaria,MIÉRCOLES,-75.564683,6.249282


In [69]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
len(df['BARRIO'].drop_duplicates())

334